In [15]:
import os
import pandas as pd

import py2neo
from py2neo import Graph,Node,Relationship

import IPython

print ('py2neo version', py2neo.__version__)

py2neo version 2021.2.3


In [16]:
db_url = 'neo4j@bolt://neo4j@localhost:7687'
db_pass = 'tweets'

graph = Graph(db_url, password = db_pass)

In [17]:
#consulta de prueba para ver que todo funcione

nro_nodos_user="MATCH(u:User) RETURN Count(u)"
nro_nodos_tweet="MATCH(t:Tweet) RETURN Count(t)"

#evaluamos las consultas
result_users=graph.evaluate(nro_nodos_user)
result_tweets=graph.evaluate(nro_nodos_tweet)

#imprimo resultados
print(f'En la BD hay {result_users} nodos User y {result_tweets} nodos Tweet')

En la BD hay 55860 nodos User y 405369 nodos Tweet


In [18]:
# Create a projection of the graph and store it on the catalog

nodos_tweet = (
    "CALL gds.graph.project('tweets', ['User', 'Tweet', 'Hashtag'], ['MENTIONS', 'TWEETED', 'TAG', 'REPLY', 'QUOTE']) "
)

#evaluamos las consultas
tweets=graph.run(nodos_tweet).to_data_frame() 

tweets

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
0,"{'Hashtag': {'label': 'Hashtag', 'properties':...","{'QUOTE': {'orientation': 'NATURAL', 'aggregat...",tweets,474112,1098318,1074


In [19]:
# Retrieve all graphs on the catalog to see if it was stored correctly

query = (
    "CALL gds.graph.list() "
)

#evaluamos las consultas
tweets=graph.run(query).to_data_frame() 

tweets

,degreeDistribution,graphName,database,memoryUsage,sizeInBytes,nodeCount,relationshipCount,configuration,density,creationTime,modificationTime,schema
0,"{'p99': 22, 'min': 0, 'max': 3502, 'mean': 2.3...",tweets,neo4j,62 MiB,65332451,474112,1098318,{'relationshipProjection': {'QUOTE': {'orienta...,0.000005,2022-06-13T18:32:47.873189000-03:00,2022-06-13T18:32:50.980297999-03:00,"{'relationships': {'QUOTE': {}, 'TAG': {}, 'ME..."


In [31]:
# Execute the PageRank centrality algorithm

query = (
    "CALL gds.pageRank.stream('tweets') "
    "YIELD nodeId, score "
    "WITH gds.util.asNode(nodeId) AS node, score "
    "WHERE node.name IS NOT NULL "
    "RETURN node.name, node.username, node.followers, node.tweet_count, score "
    "ORDER BY score DESC, node.name ASC "
    "LIMIT 20"
)

#evaluamos las consultas
tweets=graph.run(query).to_data_frame() 

tweets

,node.name,node.username,node.followers,node.tweet_count,score
0,Isabel_,Isabel66991411,5478,78777,532.363366
1,Nacional,Nacional,341359,29775,258.826929
2,☪️🅰️®️🛴🌎💰,boniatero75,2129,42533,244.961132
3,PEÑAROL,OficialCAP,434933,43753,212.705624
4,Luis Lacalle Pou,LuisLacallePou,416367,27270,212.469123
5,JACK_TORRANCE⬆️,LaMoscaFliesu2,638,13809,196.414486
6,LMP,Lelengo86,513,5257,142.805804
7,💫💫Estela❤️😷😷🏆🖤💛,EstelaF68436226,3484,84862,139.781317
8,Maria Gagliardi..@2311,mag_2311,6847,105786,130.684302
9,gaby 💖💙❤️,sgabyo,2084,28238,130.111393


In [32]:
# Execute the ArticleRank centrality algorithm

query = (
    "CALL gds.articleRank.stream('tweets') "
    "YIELD nodeId, score "
    "WITH gds.util.asNode(nodeId) AS node, score "
    "WHERE node.name IS NOT NULL "
    "RETURN node.name, node.username, node.followers, node.tweet_count, score "
    "ORDER BY score DESC, node.name ASC "
    "LIMIT 20"
)

#evaluamos las consultas
tweets=graph.run(query).to_data_frame() 

tweets

,node.name,node.username,node.followers,node.tweet_count,score
0,Isabel_,Isabel66991411,5478,78777,137.856228
1,Nacional,Nacional,341359,29775,74.258115
2,Luis Lacalle Pou,LuisLacallePou,416367,27270,68.426836
3,☪️🅰️®️🛴🌎💰,boniatero75,2129,42533,66.749522
4,PEÑAROL,OficialCAP,434933,43753,65.690199
5,JACK_TORRANCE⬆️,LaMoscaFliesu2,638,13809,59.331938
6,💫💫Estela❤️😷😷🏆🖤💛,EstelaF68436226,3484,84862,43.618648
7,EL PAÍS,elpaisuy,792004,417586,41.687620
8,ANALI BENTENCOURT,Analiros,1373,23096,40.171233
9,Graciela Bianchi,gbianchi404,74276,60786,39.878694


In [34]:
# Execute the Betweenness centrality algorithm

query = (
    "CALL gds.betweenness.stream('tweets') "
    "YIELD nodeId, score "
    "WITH gds.util.asNode(nodeId) AS node, score "
    "WHERE node.name IS NOT NULL "
    "RETURN node.name, node.username, node.followers, node.tweet_count, score "
    "ORDER BY score DESC, node.name ASC "
    "LIMIT 20"
)

#evaluamos las consultas
tweets=graph.run(query).to_data_frame() 

tweets

,node.name,node.username,node.followers,node.tweet_count,score
0,Isabel_,Isabel66991411,5478,78777,7.304500e+09
1,☪️🅰️®️🛴🌎💰,boniatero75,2129,42533,4.205308e+09
2,gaby 💖💙❤️,sgabyo,2084,28238,3.648774e+09
3,JACK_TORRANCE⬆️,LaMoscaFliesu2,638,13809,3.090501e+09
4,LMP,Lelengo86,513,5257,2.456070e+09
5,ANALI BENTENCOURT,Analiros,1373,23096,2.403511e+09
6,Jorge Andrés,JorgeAndresBusi,2554,94890,2.199247e+09
7,𝕴𝖓𝖘𝖔𝖕𝖔𝖗𝖙𝖆𝖇𝖑𝖊𝖒𝖊𝖓𝖙𝖊 𝕭𝖔𝖑𝖘𝖔,KiwiNacional,1355,53954,1.846585e+09
8,💫💫Estela❤️😷😷🏆🖤💛,EstelaF68436226,3484,84862,1.805920e+09
9,Antonella Gordillo 🇺🇾🥕,AntoGordillo,5616,14396,1.718331e+09


In [35]:
# Execute the Degree centrality algorithm

query = (
    "CALL gds.degree.stream('tweets') "
    "YIELD nodeId, score "
    "WITH gds.util.asNode(nodeId) AS node, score "
    "WHERE node.name IS NOT NULL "
    "RETURN node.name, node.username, node.followers, node.tweet_count, score "
    "ORDER BY score DESC, node.name ASC "
    "LIMIT 20"
)

#evaluamos las consultas
tweets=graph.run(query).to_data_frame() 

tweets

,node.name,node.username,node.followers,node.tweet_count,score
0,Isabel_,Isabel66991411,5478,78777,3502.0
1,JACK_TORRANCE⬆️,LaMoscaFliesu2,638,13809,2151.0
2,☪️🅰️®️🛴🌎💰,boniatero75,2129,42533,1943.0
3,gaby 💖💙❤️,sgabyo,2084,28238,1820.0
4,𝚂iL ❤️‍🔥 #36303,madrechocolates,977,77480,1646.0
5,Jenny Parada Martino,MartinoParada,1137,62402,1625.0
6,ANALI BENTENCOURT,Analiros,1373,23096,1624.0
7,LMP,Lelengo86,513,5257,1586.0
8,Cada dirección de Montevideo,everylotMVD,251,14198,1488.0
9,💫💫Estela❤️😷😷🏆🖤💛,EstelaF68436226,3484,84862,1404.0


In [36]:
# Execute the Degree centrality algorithm

query = (
    "CALL gds.louvain.stream('tweets') "
    "YIELD nodeId, communityId, intermediateCommunityIds"
    "WITH gds.util.asNode(nodeId) AS node, communityId "
    "WHERE node.name IS NOT NULL "
    "RETURN node.name, node.username, communityId "
)

#evaluamos las consultas
tweets=graph.run(query).to_data_frame() 

tweets

ClientError: [Statement.SyntaxError] Invalid input 'gds': expected
  ","
  "AS"
  "CALL"
  "CREATE"
  "DELETE"
  "DETACH"
  "FOREACH"
  "LOAD"
  "MATCH"
  "MERGE"
  "OPTIONAL"
  "REMOVE"
  "RETURN"
  "SET"
  "UNION"
  "UNWIND"
  "USE"
  "WHERE"
  "WITH"
  <EOF> (line 1, column 91 (offset: 90))
"CALL gds.louvain.stream('tweets') YIELD nodeId, communityId, intermediateCommunityIdsWITH gds.util.asNode(nodeId) AS node, communityId WHERE node.name IS NOT NULL RETURN node.name, node.username, communityId"
                                                                                           ^